In [2]:
!pip install pandas

  Using cached pandas-1.3.5-cp37-cp37m-macosx_10_9_x86_64.whl (11.0 MB)


In [10]:
import pandas as pd
data = pd.read_csv('fe_chat.csv')
print(type(data))
data.drop(columns=['No'], inplace=True)
data.dropna(inplace=True)
data.shape


<class 'pandas.core.frame.DataFrame'>


KeyError: "['No'] not found in axis"

In [9]:
data.to_csv('fe_chat.csv',index=False)
data

,Questions,Answers
0,What happens to my data?,juli users collect a vast amount of accurate p...
1,What about data privacy?,Our mission at juli is to use data and artific...
2,Do you recommend a device to use with juli?,juli uses the data collected by your iPhone. T...
3,What is Heart Rate Variability?,Heart Rate Variability is how much the time be...
4,Why doesn't juli give me any recommendations h...,"Health is complicated, and so we need enough d..."
5,What could I do to improve my asthma?,Taking your preventive medication as directed ...
6,Is there anything I need to worry about today?,Most people with asthma should focus on pollen...
7,Should I see a doctor?,If you are having specific symptoms that indic...
8,What can I do to sleep better?,If your sleep problems are asthma-induced you ...
9,Can you tell me an asthma joke?,https://laffgaff.com/asthma-jokes-puns/


In [3]:
test_data = [
    "What is the meaning of juli name",
    "Can I speak to someone at juli?",
    "Who is behind juli?",
    "Who is juli"
]

In [4]:
# master fn for getting answers to questions

def genResults(questions, fn):
    def genresult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    result_df = pd.DataFrame(list(map(genresult, questions)), columns=['question', 'question_closest', 'answer', 'score'])
    return result_df['answer'].values

## Bert QA system

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')

# def encode_questions():
#     bc = model
#     questions = data["Questions"].values.tolist()
#     print("Questions count", len(questions))
#     print("Start to calculate encoder....")
#     questions_encoder = bc.encode(questions)
#     np.save("questions", questions_encoder)
#     questions_encoder_len = np.sqrt(
#         np.sum(questions_encoder * questions_encoder, axis=1)
#     )
#     np.save("questions_len", questions_encoder_len)
#     print("Encoder ready")

# encode_questions()

In [7]:

import numpy as np

class BertAnswer():
    def __init__(self):
        self.bc = model
        self.q_data = data["Questions"].values.tolist()
        self.a_data = data["Answers"].values.tolist()
        self.questions_encoder = np.load("questions.npy")
        self.questions_encoder_len = np.load("questions_len.npy")

    def get(self, q):
        query_vector = self.bc.encode([q])[0]
        score = np.sum((query_vector * self.questions_encoder), axis=1) / (
            self.questions_encoder_len * (np.sum(query_vector * query_vector) ** 0.5)
        )
        top_id = np.argsort(score)[::-1][0]
        # if float(score[top_id]) > 0.94:
        #     return self.a_data[top_id], score[top_id], self.q_data[top_id]
        # return "Sorry, I didn't get you.", score[top_id], self.q_data[top_id]
        return self.a_data[top_id], score[top_id], self.q_data[top_id]

bm = BertAnswer()

def getBertAnswer(q):
    return bm.get(q)

genResults([test_data[0]], getBertAnswer)[0]

IndexError: list index out of range

In [ ]:
# bert approach and lrvensthein distance method both seem to be good, present results to Andy